# 🛍️ Gardrops Web Scraping - Çanta Ürünleri

Bu notebook ile Gardrops.com'dan çanta ürünlerinin detaylı bilgilerini çekip Excel'e kaydedebilirsiniz.

## 🚀 Hızlı Başlangıç

1. **Tüm hücreleri sırayla çalıştırın** (Run All)
2. Test için "TEST" bölümündeki hücreyi çalıştırın
3. Tüm ürünler için "TÜM ÜRÜNLERİ ÇEK" hücresini çalıştırın  
4. "EXCEL'E KAYDET" hücresini çalıştırarak dosyayı kaydedin

## 📊 Çekilen Bilgiler
- Ürün Adı, Fiyat, Marka, Durum, Renk
- Açıklama, Satıcı, Kategori
- Beğeni Sayısı, Görüntülenme
- Ürün URL'si

## ⚙️ Teknolojiler
- **Selenium**: Dinamik içerik render
- **BeautifulSoup**: HTML parsing
- **Pandas**: Veri işleme
- **OpenPyXL**: Excel çıktısı

---

In [23]:
# Gardrops Web Scraping - Çanta Ürünleri
# Gerekli kütüphaneleri yükle
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urljoin
import re
import os
import json

In [24]:
# Ana URL ve headers
base_url = "https://www.gardrops.com/search?subCats=1408%2C1402%2C1403%2C1410%2C1405%2C1409%2C1404%2C1407%2C1413%2C1406%2C1411&page=1"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

In [57]:
# Ürün detaylarını çekme fonksiyonu (GELİŞMİŞ - Selenium ile + Görsel)
def get_product_details(product_url):
    """Ürün detay sayfasından bilgileri çeker - Selenium ile dinamik içerik"""
    
    product_data = {
        'URL': product_url,
        'Ürün Adı': '',
        'Fiyat': '',
        'Marka': '',
        'Kullanım Durumu': '',
        'Renk': '',
        'Kategori': '',
        'İlan Tarihi': '',
        'Açıklama': '',
        'Satıcı': '',
        'Görsel URL': '',
        'Beğeni Sayısı': '',
        'Görüntülenme': ''
    }
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    try:
        driver.get(product_url)
        
        # Sayfa tam yüklenene kadar bekle
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        time.sleep(3)  # Ekstra bekleme
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # HTML'den direkt regex ile çıkar (Next.js escaped JSON)
        page_html = driver.page_source
        
        # Fiyat
        price_match = re.search(r'\\"price\\":(\d+)', page_html)
        if price_match:
            product_data['Fiyat'] = price_match.group(1) + ' ₺'
        
        # Marka
        brand_match = re.search(r'\\"brandName\\":\\"([^"\\\\]+)\\"', page_html)
        if brand_match:
            product_data['Marka'] = brand_match.group(1)
        
        # Kategori
        category_match = re.search(r'\{\\"type\\":\\"CATEGORY\\",\\"key\\":\\"Kategori\\",\\"value\\":\\"([^"\\\\]+)\\"\}', page_html)
        if category_match:
            product_data['Kategori'] = category_match.group(1)
        
        # Kullanım Durumu
        condition_match = re.search(r'\{\\"type\\":\\"CONDITION\\",\\"key\\":\\"Kullanım Durumu\\",\\"value\\":\\"([^"\\\\]+)\\"\}', page_html)
        if condition_match:
            product_data['Kullanım Durumu'] = condition_match.group(1)
        
        # Renk
        color_match = re.search(r'\{\\"type\\":\\"COLOR\\",\\"key\\":\\"Renk\\",\\"value\\":\\"([^"\\\\]+)\\"\}', page_html)
        if color_match:
            product_data['Renk'] = color_match.group(1)
        
        # İlan Tarihi
        date_match = re.search(r'\{\\"type\\":\\"UPLOAD_DATE\\",\\"key\\":\\"Yüklenme Tarihi\\",\\"value\\":\\"([^"\\\\]+)\\"\}', page_html)
        if date_match:
            product_data['İlan Tarihi'] = date_match.group(1)
        
        # Görsel URL
        image_match = re.search(r'\\"images\\":\[\{\\"small\\":\\"([^"\\\\]+)\\"', page_html)
        if image_match:
            product_data['Görsel URL'] = image_match.group(1)
        
        # Satıcı
        seller_match = re.search(r'\\"username\\":\\"([^"\\\\]+)\\"', page_html)
        if seller_match:
            product_data['Satıcı'] = seller_match.group(1)
        
        # Beğeni
        like_match = re.search(r'\\"likeCount\\":(\d+)', page_html)
        if like_match:
            product_data['Beğeni Sayısı'] = like_match.group(1)
        
        # Ürün başlığı ve açıklama
        title_match = re.search(r'\\"title\\":\\"([^"\\\\]+)\\"', page_html)
        if title_match:
            product_data['Ürün Adı'] = title_match.group(1)
        
        description_match = re.search(r'\\"description\\":\\"([^"\\\\]+)\\"', page_html)
        if description_match:
            product_data['Açıklama'] = description_match.group(1)
        
        # Eğer JSON'dan alınamayanlar varsa, fallback olarak text parse et
        if not product_data['Ürün Adı']:
            meta_title = soup.find('meta', property='og:title')
            if meta_title:
                product_data['Ürün Adı'] = meta_title.get('content', '')
        
        if not product_data['Açıklama']:
            meta_desc = soup.find('meta', property='og:description')
            if meta_desc:
                product_data['Açıklama'] = meta_desc.get('content', '')
        
        if not product_data['Görsel URL']:
            image_link = soup.find('link', rel='preload', attrs={'as': 'image'})
            if image_link:
                product_data['Görsel URL'] = image_link.get('href', '')
        
        return product_data
    
    except Exception as e:
        print(f"✗ Ürün detayı çekilemedi ({product_url}): {e}")
        return None
    
    finally:
        driver.quit()

In [64]:
# 🚀 TÜM 50 SAYFAYI TARA VE EXCEL'E KAYDET
from datetime import datetime

print("🚀 TÜM SAYFALARIN SCRAPING'İ BAŞLIYOR...")
print("="*80)
print("⏰ Bu işlem 2-4 saat sürebilir!")
print("📊 ~50 sayfa x ~70 ürün = ~3500 ürün bekleniyor")
print("="*80 + "\n")

# Base URL (page=1 parametresi önemli - fonksiyon bunu değiştirecek)
base_url = "https://www.gardrops.com/search?subCats=1408%2C1402%2C1403%2C1410%2C1405%2C1409%2C1404%2C1407%2C1413%2C1406%2C1411&page=1"

# Tüm 50 sayfayı tara
all_products_50_pages = scrape_gardrops_with_selenium(
    base_url=base_url,
    max_pages=50,  # 50 sayfa
    max_products_per_page=None  # Her sayfadaki tüm ürünler
)

if all_products_50_pages and len(all_products_50_pages) > 0:
    # DataFrame'e çevir
    df_all_pages = pd.DataFrame(all_products_50_pages)
    
    # Excel'e kaydet
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"gardrops_cantalar_50_SAYFA_{timestamp}.xlsx"
    
    df_all_pages.to_excel(filename, index=False, engine='openpyxl')
    
    print("\n" + "="*80)
    print(f"🎉 BAŞARILI! TÜM VERİ KAYDEDİLDİ!")
    print("="*80)
    print(f"✅ Toplam ürün sayısı: {len(df_all_pages)}")
    print(f"📁 Dosya adı: {filename}")
    print(f"📍 Konum: {os.path.abspath(filename)}")
    print("="*80)
    
    # İstatistikler
    print("\n📊 İSTATİSTİKLER:")
    print(f"  • Benzersiz marka sayısı: {df_all_pages['Marka'].nunique()}")
    print(f"  • Benzersiz kategori sayısı: {df_all_pages['Kategori'].nunique()}")
    print(f"  • Ortalama fiyat: {df_all_pages['Fiyat'].str.replace(' ₺', '').astype(float).mean():.2f} ₺")
    
    # İlk 5 ürünü göster
    print("\n📋 İLK 5 ÜRÜN:")
    print(df_all_pages.head().to_string())
    
else:
    print("\n❌ Hata: Hiç ürün çekilemedi!")

🚀 TÜM SAYFALARIN SCRAPING'İ BAŞLIYOR...
⏰ Bu işlem 2-4 saat sürebilir!
📊 ~50 sayfa x ~70 ürün = ~3500 ürün bekleniyor

🚀 ÇOK SAYFALIK SELENIUM SCRAPING BAŞLIYOR...

📄 Taranacak sayfa sayısı: 50


📄 SAYFA 1/50
📍 URL: https://www.gardrops.com/search?subCats=1408%2C1402%2C1403%2C1410%2C1405%2C1409%2C1404%2C1407%2C1413%2C1406%2C1411&page=1
⏳ Sayfa yükleniyor...
📜 Sayfa kaydırılıyor...
🔍 Ürün linkleri aranıyor...
✅ Sayfa 1: 71 benzersiz ürün linki bulundu
📦 Sayfa 1: 71 ürünün detayları çekiliyor...
   Sayfa 1 - Ürün 71/71: https://www.gardrops.com/0-lumberjack-bej-fermuarl...
✅ Sayfa 1 tamamlandı: 71 ürün çekildi (Toplam: 71)

📄 SAYFA 2/50
📍 URL: https://www.gardrops.com/search?subCats=1408%2C1402%2C1403%2C1410%2C1405%2C1409%2C1404%2C1407%2C1413%2C1406%2C1411&page=2
⏳ Sayfa yükleniyor...
📜 Sayfa kaydırılıyor...
🔍 Ürün linkleri aranıyor...
✅ Sayfa 2: 71 benzersiz ürün linki bulundu
📦 Sayfa 2: 71 ürünün detayları çekiliyor...
   Sayfa 2 - Ürün 71/71: https://www.gardrops.com/kahverengi-kadin-

In [65]:
# 📊 SONUÇLARI GÖSTER
print("\n" + "="*80)
print("📊 SCRAPING TAMAMLANDI - SONUÇLAR")
print("="*80)

if 'df_all_pages' in locals() and not df_all_pages.empty:
    print(f"\n✅ Toplam Ürün Sayısı: {len(df_all_pages)}")
    print(f"📁 Dosya: gardrops_cantalar_50_SAYFA_*.xlsx")
    print(f"📏 Dosya Boyutu: ~599 KB")
    
    print(f"\n📊 Detaylı İstatistikler:")
    print(f"  • Toplam Ürün: {len(df_all_pages)}")
    print(f"  • Benzersiz Marka: {df_all_pages['Marka'].nunique()}")
    print(f"  • Benzersiz Kategori: {df_all_pages['Kategori'].nunique()}")
    print(f"  • Benzersiz Satıcı: {df_all_pages['Satıcı'].nunique()}")
    
    # Fiyat istatistikleri
    fiyatlar = df_all_pages['Fiyat'].str.replace(' ₺', '').str.replace(',', '').astype(float)
    print(f"\n💰 Fiyat İstatistikleri:")
    print(f"  • Minimum: {fiyatlar.min():.0f} ₺")
    print(f"  • Maksimum: {fiyatlar.max():.0f} ₺")
    print(f"  • Ortalama: {fiyatlar.mean():.2f} ₺")
    print(f"  • Medyan: {fiyatlar.median():.0f} ₺")
    
    # En popüler markalar
    print(f"\n🏆 En Popüler 5 Marka:")
    top_brands = df_all_pages['Marka'].value_counts().head(5)
    for idx, (marka, count) in enumerate(top_brands.items(), 1):
        print(f"  {idx}. {marka}: {count} ürün")
    
    # En popüler kategoriler
    print(f"\n📦 Kategori Dağılımı:")
    top_cats = df_all_pages['Kategori'].value_counts().head(5)
    for idx, (kat, count) in enumerate(top_cats.items(), 1):
        print(f"  {idx}. {kat}: {count} ürün")
    
    # İlk 3 ürün
    print(f"\n📋 İlk 3 Ürün Örneği:")
    print(df_all_pages[['Ürün Adı', 'Fiyat', 'Marka', 'Kategori']].head(3).to_string(index=False))
    
    print("\n" + "="*80)
    print("✅ Tüm veriler başarıyla çekildi ve Excel'e kaydedildi!")
    print("="*80)
else:
    print("❌ Veri bulunamadı!")


📊 SCRAPING TAMAMLANDI - SONUÇLAR

✅ Toplam Ürün Sayısı: 3574
📁 Dosya: gardrops_cantalar_50_SAYFA_*.xlsx
📏 Dosya Boyutu: ~599 KB

📊 Detaylı İstatistikler:
  • Toplam Ürün: 3574
  • Benzersiz Marka: 144
  • Benzersiz Kategori: 11
  • Benzersiz Satıcı: 821

💰 Fiyat İstatistikleri:
  • Minimum: 20 ₺
  • Maksimum: 25000 ₺
  • Ortalama: 747.01 ₺
  • Medyan: 400 ₺

🏆 En Popüler 5 Marka:
  1. Diğer: 1637 ürün
  2. Zara: 288 ürün
  3. Vakko: 136 ürün
  4. Guess: 118 ürün
  5. Mango: 115 ürün

📦 Kategori Dağılımı:
  1. Kol Çantası: 2664 ürün
  2. Çapraz Çanta: 300 ürün
  3. Cüzdan: 246 ürün
  4. Sırt Çantası: 147 ürün
  5. Clutch / Portföy: 112 ürün

📋 İlk 3 Ürün Örneği:
                        Ürün Adı Fiyat        Marka     Kategori
        Calvin Klein Siyah Çanta 400 ₺ Calvin Klein Çapraz Çanta
     Kahverengi Örgü Kadın Çanta 800 ₺        Diğer  Kol Çantası
Adidas Kırmızı Spor Sırt Çantası 300 ₺       Adidas Sırt Çantası

✅ Tüm veriler başarıyla çekildi ve Excel'e kaydedildi!


In [62]:
# SELENIUM İLE KOMPLE SCRAPING ÇÖZÜMÜ - TÜM SAYFALAR
import json

def scrape_gardrops_with_selenium(base_url, max_pages=50, max_products_per_page=None):
    """
    Selenium ile Gardrops'tan tüm sayfaları tarayarak ürün bilgilerini çeker
    
    Args:
        base_url: Arama sayfası URL'i (page=1 içermeli)
        max_pages: Taranacak maksimum sayfa sayısı (varsayılan: 50)
        max_products_per_page: Her sayfadan çekilecek max ürün sayısı (None = hepsi)
    
    Returns:
        list: Tüm ürünlerin bilgilerini içeren liste
    """
    print("🚀 ÇOK SAYFALIK SELENIUM SCRAPING BAŞLIYOR...\n")
    print("="*80)
    print(f"📄 Taranacak sayfa sayısı: {max_pages}")
    print("="*80 + "\n")
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    all_products = []
    
    try:
        # Tüm sayfaları tara
        for page_num in range(1, max_pages + 1):
            # URL'deki page parametresini güncelle
            current_url = base_url.replace('page=1', f'page={page_num}')
            
            print(f"\n{'='*80}")
            print(f"📄 SAYFA {page_num}/{max_pages}")
            print(f"📍 URL: {current_url}")
            print(f"{'='*80}")
            
            driver.get(current_url)
            
            # Sayfanın yüklenmesini bekle
            print("⏳ Sayfa yükleniyor...")
            time.sleep(8)  # JavaScript'in render olması için yeterli süre
        
            # Sayfayı aşağı kaydır (lazy loading için)
            print("📜 Sayfa kaydırılıyor...")
            for i in range(3):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
            
            # Sayfayı tekrar yukarı getir
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(2)
            
            # Render edilmiş HTML'i al
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Ürün linklerini bul - Gardrops'ta ürünler <a> tag'leri içinde
            print("🔍 Ürün linkleri aranıyor...")
            
            # Link bulma stratejisi: img tag'li linkleri bul (ürün görselleri)
            all_links = soup.find_all('a', href=True)
            product_links = []
            
            for link in all_links:
                href = link.get('href', '')
                # Ürün linki pattern'i: Görsel içeren ve kategori/filtre olmayan linkler
                if href.startswith('/') and link.find('img'):
                    # Exclude patterns
                    if not any(x in href.lower() for x in ['premium', 'help', 'download', 'about', 
                                                            'kadin-x', 'erkek-x', 'cocuk-x', '-x-c', 
                                                            '-x-sc', 'cart', 'favorites', 'search']):
                        full_url = urljoin('https://www.gardrops.com', href)
                        if full_url not in product_links:
                            product_links.append(full_url)
            
            print(f"✅ Sayfa {page_num}: {len(product_links)} benzersiz ürün linki bulundu")
            
            if not product_links:
                print(f"⚠️  Sayfa {page_num}'de ürün bulunamadı. Bir sonraki sayfaya geçiliyor...")
                continue
            
            if max_products_per_page:
                product_links = product_links[:max_products_per_page]
            
            # Bu sayfadaki her ürünün detaylarını çek
            print(f"📦 Sayfa {page_num}: {len(product_links)} ürünün detayları çekiliyor...")
            
            page_products = 0
            for idx, product_url in enumerate(product_links, 1):
                print(f"   Sayfa {page_num} - Ürün {idx}/{len(product_links)}: {product_url[:50]}...", end='\r')
                
                try:
                    product_data = get_product_details(product_url)
                    if product_data and product_data['Ürün Adı']:  # Sadece geçerli ürünleri ekle
                        all_products.append(product_data)
                        page_products += 1
                    time.sleep(1)  # Rate limiting
                except Exception as e:
                    print(f"\n   ⚠️  Ürün atlandı: {str(e)[:50]}")
                    continue
            
            print(f"\n✅ Sayfa {page_num} tamamlandı: {page_products} ürün çekildi (Toplam: {len(all_products)})")
        
        print(f"\n\n{'='*80}")
        print(f"🎉 TÜM SAYFALAR TAMAMLANDI!")
        print(f"✅ Toplam {len(all_products)} ürün {max_pages} sayfadan başarıyla çekildi!")
        print(f"{'='*80}")
        
    except Exception as e:
        print(f"\n❌ Hata: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        driver.quit()
        print("\n🔚 Tarayıcı kapatıldı")
    
    return all_products